In [ ]:
import pandas as pd
import numpy as np
data=pd.read_csv('')

data.drop(['id','Unnamed: 32'],axis=1,inplace=True)
data.info()

In [ ]:
data_numeric_columns=data.select_dtypes(include=['int64','float64']).columns.tolist()
data_numeric_columns
#to select categorical columns use
#data_cat_columns=data.select_dtypes(include=['object']).columns.tolist()
data[data_numeric_columns]

In [ ]:
X=data.drop('diagnosis',axis=1)#features
Y=data['diagnosis'] #target column

In [ ]:
#step 1 standardize the feature
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_scaled=sc.fit_transform(X)


In [ ]:
from sklearn.decomposition import PCA


In [ ]:
X.shape

In [ ]:
pca=PCA()
pca.fit(X_scaled)
#calculate the explained variance ratio and its cumulative sum
cumulative_variance=pca.explained_variance_ratio_.cumsum()
n_components=(cumulative_variance>=0.95).argmax()+1
n_components,cumulative_variance

In [ ]:
import matplotlib.pyplot as plt
explained_variance_ratio_cancer=pca.explained_variance_ratio_
plt.figure(figsize=(18,8))
plt.plot(np.cumsum(explained_variance_ratio_cancer),marker='o',
         linestyle='--',label='Explained Variance Ratio cumsum')
plt.bar(range(1,len(explained_variance_ratio_cancer)+1),
        explained_variance_ratio_cancer,
        alpha=0.9,label='Indivudual Component score')
plt.title('Scree Plot')
plt.xlabel('No. of Components')
plt.ylabel('Explained variance ratio')
plt.axhline(y=0.95,color='r',linestyle='--',
            label='95% of the Explained variance ratio')
plt.xticks(range(1,len(explained_variance_ratio_cancer)+1))
plt.legend()
plt.grid()
plt.show()

In [ ]:
pca=PCA(n_components=11)#no. of components +1 >0.95 10 components
pca.fit(X_scaled)
prop_var=pca.explained_variance_ratio_
n_components=np.arange(pca.n_components_)+1



In [ ]:
plt.plot(n_components,prop_var,'ro-')
plt.show()

In [ ]:
#the optimal n_components is 2 as per scree plot
pca_final=PCA(n_components=2)
pca_components=pca_final.fit_transform(X_scaled)

In [ ]:
loading=pca_final.components_
feature_names=X.columns
pc1_loading=pd.Series(loading[0],index=feature_names)
pc1_loading_sort=pc1_loading.abs().sort_values(ascending=False)
print('PC1',pc1_loading_sort)
pc2_loading=pd.Series(loading[1],index=feature_names)
pc2_loading_sort=pc2_loading.abs().sort_values(ascending=False)
print('PC2',pc2_loading_sort)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(18,8))
data_final=pd.DataFrame(data=pca_components,columns=['PC1','PC2'])
finaldf=pd.concat([data_final,Y],axis=1)
finaldf
sns.scatterplot(x=finaldf['PC1'],y=finaldf['PC2'],hue=finaldf['diagnosis'],
                palette='Set1')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
finaldf.info()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
X=finaldf.drop('diagnosis',axis=1)
Y=finaldf['diagnosis']
lbl=LabelEncoder()
y_encoded=lbl.fit_transform(Y)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X_train,X_test,Y_train,Y_test=train_test_split(X,y_encoded,test_size=0.2,
                                               random_state=42)


In [ ]:
rnd=RandomForestClassifier(n_estimators=100,random_state=42)
rnd.fit(X_train,Y_train)
y_pred=rnd.predict(X_test)
print(classification_report(Y_test,y_pred))